# Unit 8c — Design Patterns

**Purpose:** Learn common design patterns that solve recurring software design problems. These patterns represent proven solutions used throughout industry software development.

---

## 📚 Table of Contents

1. [Composition vs Inheritance](#1.-Composition-vs-Inheritance)
2. [Strategy Pattern](#2.-Strategy-Pattern)
3. [Factory Pattern](#3.-Factory-Pattern)
4. [Singleton Pattern](#4.-Singleton-Pattern)
5. [Exercises](#5.-Exercises)

---

## Prerequisites

Before starting this unit, you should be comfortable with:
- Unit 7 (OOP Foundations) — classes, objects, inheritance basics
- Unit 8a (Advanced OOP Techniques) — ABCs, type hints, properties
- Unit 8b (SOLID Principles) — especially OCP and DIP

## 🎯 Learning Objectives

By the end of this unit, you will be able to:

**🏗️ Composition vs Inheritance**
- Choose between composition ("has-a") and inheritance ("is-a") relationships
- Understand why composition is often preferred over inheritance
- Refactor inheritance hierarchies to use composition

**🎯 Strategy Pattern**
- Encapsulate interchangeable algorithms in separate classes
- Change behavior at runtime by swapping strategy objects
- Apply strategy pattern to real-world problems

**🏭 Factory Pattern**
- Centralize object creation logic
- Use registry pattern for extensible factories
- Decouple client code from concrete class instantiation

**🔒 Singleton Pattern**
- Ensure a class has only one instance
- Understand singleton trade-offs and when to use alternatives
- Implement Pythonic singleton patterns

In [ ]:
# Setup: Import common modules
from abc import ABC, abstractmethod
import json

---

# 1. Composition vs Inheritance

## 1.1 Choosing the Right Relationship

When designing relationships between classes, consider:

| Question | Answer | Relationship |
|----------|--------|-------------|
| Is A **a type of** B? | Yes | Inheritance (is-a) |
| Does A **have** or **use** B? | Yes | Composition (has-a) |

### Examples

| Relationship | Type | Reasoning |
|--------------|------|----------|
| Dog → Animal | Inheritance | A dog IS an animal |
| Car → Engine | Composition | A car HAS an engine |
| Manager → Employee | Inheritance | A manager IS an employee |
| Company → Employee | Composition | A company HAS employees |

## 1.2 Prefer Composition Over Inheritance

> "Favor composition over inheritance" — *Design Patterns* (Gang of Four)

**Problems with Inheritance:**
- Tight coupling between parent and child
- Changes to parent can break children
- Deep hierarchies become hard to understand
- Behavior cannot change at runtime

**Benefits of Composition:**
- Loose coupling
- Components can be swapped at runtime
- Easier to test (mock components)
- More flexible combinations

## 1.5 The Inheritance Trap

### Why Inheritance Can Be Problematic

Inheritance creates a strong **compile-time** relationship. Consider this evolution:

```
Version 1: Bird → can fly()
Version 2: Add Penguin (Bird) — but penguins can't fly!
Version 3: Add Ostrich (Bird) — also can't fly!
Version 4: Refactor entire hierarchy...
```

This is called the **Fragile Base Class Problem** — changes to parent classes ripple through all children.

### The Diamond Problem

Multiple inheritance can create ambiguity:

```
       Animal
      /      \
   Flyer    Swimmer
      \      /
       Duck
```

Which `Animal` does `Duck` inherit from? Python uses MRO (Method Resolution Order), but the complexity remains.

### Composition Solves These Problems

```python
# Instead of inheritance hierarchies:
class Duck:
    def __init__(self):
        self.flying = FlyingBehavior()    # Can fly
        self.swimming = SwimmingBehavior() # Can swim
    
    def fly(self):
        return self.flying.perform()
    
    def swim(self):
        return self.swimming.perform()

class Penguin:
    def __init__(self):
        self.flying = NoFlyBehavior()      # Can't fly
        self.swimming = SwimmingBehavior() # Can swim
```

### Decision Framework

```
┌─────────────────────────────────────────────────────────┐
│           Should I Use Inheritance?                      │
├─────────────────────────────────────────────────────────┤
│                                                          │
│  Is it a TRUE "is-a" relationship?                      │
│       │                                                  │
│       ├── No ──► Use Composition                        │
│       │                                                  │
│       ▼ Yes                                             │
│                                                          │
│  Will the subclass ALWAYS be substitutable (LSP)?       │
│       │                                                  │
│       ├── No ──► Use Composition                        │
│       │                                                  │
│       ▼ Yes                                             │
│                                                          │
│  Is the hierarchy shallow (≤3 levels)?                  │
│       │                                                  │
│       ├── No ──► Refactor with Composition              │
│       │                                                  │
│       ▼ Yes                                             │
│                                                          │
│  ✅ Inheritance is appropriate                          │
│                                                          │
└─────────────────────────────────────────────────────────┘
```

## 1.3 Composition Example: Report with Exporters

In [ ]:
# Composition: Report HAS an exporter (can be swapped)

class Exporter(ABC):
    """Interface for exporters."""
    
    @abstractmethod
    def export(self, title: str, data: dict) -> str:
        pass


class TextExporter(Exporter):
    """Export as plain text."""
    
    def export(self, title: str, data: dict) -> str:
        lines = [f"=== {title} ===", "-" * 30]
        for key, value in data.items():
            lines.append(f"  {key}: {value}")
        lines.append("-" * 30)
        return "\n".join(lines)


class JSONExporter(Exporter):
    """Export as JSON."""
    
    def export(self, title: str, data: dict) -> str:
        return json.dumps({"title": title, "data": data}, indent=2)


class HTMLExporter(Exporter):
    """Export as HTML."""
    
    def export(self, title: str, data: dict) -> str:
        rows = "".join(f"<tr><td>{k}</td><td>{v}</td></tr>" for k, v in data.items())
        return f"<h1>{title}</h1><table>{rows}</table>"


class Report:
    """Report uses composition — exporter can be changed at runtime."""
    
    def __init__(self, title: str, data: dict, exporter: Exporter):
        self.title = title
        self.data = data
        self.exporter = exporter  # Composition: Report HAS an exporter
    
    def export(self) -> str:
        return self.exporter.export(self.title, self.data)
    
    def change_exporter(self, new_exporter: Exporter) -> None:
        """Change format at runtime — only possible with composition!"""
        self.exporter = new_exporter


# Create report with text exporter
report = Report("Sales Summary", {"revenue": 50000, "units": 1200}, TextExporter())
print(report.export())
print()

# Switch to JSON at runtime!
report.change_exporter(JSONExporter())
print(report.export())
print()

# Switch to HTML
report.change_exporter(HTMLExporter())
print(report.export())

## 1.4 When Inheritance Is Appropriate

Inheritance works well when:
- There is a genuine "is-a" relationship
- Subclasses truly specialize the parent
- The hierarchy is shallow (2–3 levels maximum)
- Polymorphism is needed (treating different types uniformly)

In [ ]:
# Good use of inheritance: Account types

class BankAccountBase:
    """Base account with common functionality."""
    
    def __init__(self, owner: str, balance: float = 0.0):
        self.owner = owner
        self._balance = float(balance)

    def deposit(self, amount: float) -> float:
        if amount <= 0:
            raise ValueError("Amount must be positive")
        self._balance += float(amount)
        return self._balance
    
    def withdraw(self, amount: float) -> float:
        if amount <= 0:
            raise ValueError("Amount must be positive")
        if amount > self._balance:
            raise ValueError("Insufficient funds")
        self._balance -= float(amount)
        return self._balance

    @property
    def balance(self) -> float:
        return self._balance
    
    def __repr__(self) -> str:
        return f"{self.__class__.__name__}(owner={self.owner}, balance={self._balance})"


class SavingsAccount(BankAccountBase):
    """Savings account: earns interest."""
    
    def __init__(self, owner: str, balance: float = 0.0, interest_rate: float = 0.02):
        super().__init__(owner, balance)
        self.interest_rate = interest_rate
    
    def apply_interest(self) -> float:
        """Apply monthly interest."""
        interest = self._balance * self.interest_rate
        self._balance += interest
        return interest


class CheckingAccount(BankAccountBase):
    """Checking account: has overdraft protection."""
    
    def __init__(self, owner: str, balance: float = 0.0, overdraft_limit: float = 100):
        super().__init__(owner, balance)
        self.overdraft_limit = overdraft_limit
    
    def withdraw(self, amount: float) -> float:
        """Override: allows overdraft up to limit."""
        if amount <= 0:
            raise ValueError("Amount must be positive")
        if amount > self._balance + self.overdraft_limit:
            raise ValueError(f"Exceeds overdraft limit (max: ${self._balance + self.overdraft_limit})")
        self._balance -= float(amount)
        return self._balance


# Polymorphism: treat all accounts uniformly
accounts: list[BankAccountBase] = [
    SavingsAccount("Alice", 1000, interest_rate=0.03),
    CheckingAccount("Bob", 500, overdraft_limit=200),
]

# Common operations work on all account types
for acc in accounts:
    acc.deposit(100)
    print(acc)

# Type-specific operations
savings = accounts[0]
if isinstance(savings, SavingsAccount):
    interest_earned = savings.apply_interest()
    print(f"\nInterest earned: ${interest_earned:.2f}")

checking = accounts[1]
if isinstance(checking, CheckingAccount):
    checking.withdraw(650)  # Uses overdraft!
    print(f"Checking after overdraft: {checking}")

---

# 2. Strategy Pattern

## 2.1 Overview

The **Strategy Pattern** defines a family of algorithms, encapsulates each one, and makes them interchangeable. Strategy lets the algorithm vary independently from clients that use it.

### When to Use Strategy

| Scenario | Example |
|----------|--------|
| Multiple algorithms for same task | Sorting: quicksort, mergesort, heapsort |
| Behavior changes at runtime | Payment: credit card, PayPal, crypto |
| Avoid complex conditionals | Shipping: standard, express, overnight |
| Different formatting options | Export: JSON, XML, CSV |

### Structure

1. **Strategy interface** (ABC): Defines the algorithm interface
2. **Concrete strategies**: Implement specific algorithms
3. **Context**: Uses a strategy object, delegates work to it

## 2.2 Strategy Example: Payment Processing

## 2.4 Strategy Pattern Deep Dive

### What is a Design Pattern?

Design patterns are **reusable solutions** to common software design problems. They are:
- Not code you copy-paste, but templates you adapt
- Language-independent concepts (though implementation varies)
- Documented with: intent, structure, participants, consequences

The "Gang of Four" (GoF) book *Design Patterns* (1994) catalogued 23 patterns in three categories:

| Category | Purpose | Examples |
|----------|---------|----------|
| **Creational** | Object creation | Factory, Singleton, Builder |
| **Structural** | Class composition | Adapter, Decorator, Facade |
| **Behavioral** | Object interaction | Strategy, Observer, Command |

### Strategy Pattern Structure

```
┌─────────────────────────────────────────────────────────┐
│                    Strategy Pattern                      │
├─────────────────────────────────────────────────────────┤
│                                                          │
│   ┌─────────────┐         ┌──────────────────┐          │
│   │   Context   │────────►│ Strategy (ABC)   │          │
│   │             │         │                  │          │
│   │ - strategy  │         │ + algorithm()    │          │
│   │ + execute() │         └──────────────────┘          │
│   └─────────────┘                  ▲                    │
│                                    │                    │
│         ┌──────────────────────────┼──────────────┐     │
│         │                          │              │     │
│  ┌──────┴──────┐   ┌───────────────┴┐  ┌─────────┴──┐  │
│  │ ConcreteA   │   │  ConcreteB     │  │ ConcreteC  │  │
│  │             │   │                │  │            │  │
│  │ +algorithm()│   │  +algorithm()  │  │+algorithm()│  │
│  └─────────────┘   └────────────────┘  └────────────┘  │
│                                                          │
└─────────────────────────────────────────────────────────┘
```

### Strategy vs. Conditional Logic

| Approach | Pros | Cons |
|----------|------|------|
| **if/elif chain** | Simple for 2-3 cases | Grows unwieldy, violates OCP |
| **Strategy pattern** | Extensible, testable | More classes, initial overhead |

**Rule of thumb:** Use strategy when you have 3+ algorithms or expect growth.

### Strategy Pattern and SOLID

The Strategy pattern naturally supports multiple SOLID principles:

| Principle | How Strategy Supports It |
|-----------|-------------------------|
| **SRP** | Each strategy has one algorithm |
| **OCP** | Add strategies without changing context |
| **LSP** | All strategies are interchangeable |
| **DIP** | Context depends on abstraction |

### Real-World Strategy Examples

| Domain | Context | Strategies |
|--------|---------|------------|
| **E-commerce** | Checkout | PayPal, Credit Card, Crypto |
| **Compression** | File archiver | ZIP, GZIP, BZIP2, LZ4 |
| **Routing** | Navigation app | Fastest, Shortest, Scenic |
| **Authentication** | Login system | Password, OAuth, 2FA, Biometric |
| **Sorting** | Data processor | QuickSort, MergeSort, TimSort |

In [ ]:
# Strategy Pattern: Payment processing

class PaymentStrategy(ABC):
    """Strategy interface for payment methods."""
    
    @abstractmethod
    def pay(self, amount: float) -> bool:
        """Process payment, return True if successful."""
        pass
    
    @abstractmethod
    def name(self) -> str:
        """Return the payment method name."""
        pass


class CreditCardPayment(PaymentStrategy):
    """Concrete strategy: Credit card payment."""
    
    def __init__(self, card_number: str, cvv: str):
        self.card_number = card_number[-4:]  # Store only last 4 digits
        self.cvv = cvv
    
    def pay(self, amount: float) -> bool:
        print(f"Processing ${amount:.2f} via Credit Card ending in {self.card_number}...")
        return True  # Simulated success
    
    def name(self) -> str:
        return f"Credit Card (****{self.card_number})"


class PayPalPayment(PaymentStrategy):
    """Concrete strategy: PayPal payment."""
    
    def __init__(self, email: str):
        self.email = email
    
    def pay(self, amount: float) -> bool:
        print(f"Processing ${amount:.2f} via PayPal ({self.email})...")
        return True
    
    def name(self) -> str:
        return f"PayPal ({self.email})"


class BitcoinPayment(PaymentStrategy):
    """Concrete strategy: Bitcoin payment."""
    
    def __init__(self, wallet_address: str):
        self.wallet = wallet_address[:8] + "..."  # Truncate for display
    
    def pay(self, amount: float) -> bool:
        print(f"Processing ${amount:.2f} via Bitcoin to {self.wallet}...")
        return True
    
    def name(self) -> str:
        return f"Bitcoin ({self.wallet})"


class ShoppingCart:
    """Context: Uses a payment strategy."""
    
    def __init__(self):
        self.items: list[tuple[str, float]] = []
        self.payment_strategy: PaymentStrategy | None = None
    
    def add_item(self, name: str, price: float) -> None:
        self.items.append((name, price))
    
    def total(self) -> float:
        return sum(price for _, price in self.items)
    
    def set_payment_method(self, strategy: PaymentStrategy) -> None:
        """Change payment method at runtime."""
        self.payment_strategy = strategy
        print(f"Payment method set to: {strategy.name()}")
    
    def checkout(self) -> bool:
        """Process checkout with current payment strategy."""
        if not self.payment_strategy:
            raise ValueError("No payment method set!")
        
        amount = self.total()
        print(f"\nCheckout total: ${amount:.2f}")
        return self.payment_strategy.pay(amount)


# Usage
cart = ShoppingCart()
cart.add_item("Laptop", 999.99)
cart.add_item("Mouse", 29.99)

# Try different payment strategies
print("=== Credit Card ===")
cart.set_payment_method(CreditCardPayment("4111111111111111", "123"))
cart.checkout()

print("\n=== PayPal ===")
cart.set_payment_method(PayPalPayment("user@example.com"))
cart.checkout()

print("\n=== Bitcoin ===")
cart.set_payment_method(BitcoinPayment("1A1zP1eP5QGefi2DMPTfTL5SLmv7DivfNa"))
cart.checkout()

## 2.3 Strategy with Shipping Options

In [ ]:
# Another Strategy example: Shipping calculation

class ShippingStrategy(ABC):
    """Strategy interface for shipping calculation."""
    
    @abstractmethod
    def calculate(self, weight: float, distance: float) -> float:
        """Calculate shipping cost."""
        pass
    
    @property
    @abstractmethod
    def delivery_days(self) -> int:
        """Estimated delivery time in days."""
        pass


class StandardShipping(ShippingStrategy):
    def calculate(self, weight: float, distance: float) -> float:
        return weight * 0.5 + distance * 0.01
    
    @property
    def delivery_days(self) -> int:
        return 7


class ExpressShipping(ShippingStrategy):
    def calculate(self, weight: float, distance: float) -> float:
        return weight * 1.0 + distance * 0.05
    
    @property
    def delivery_days(self) -> int:
        return 3


class OvernightShipping(ShippingStrategy):
    def calculate(self, weight: float, distance: float) -> float:
        return weight * 2.0 + distance * 0.1 + 10.0  # Base fee
    
    @property
    def delivery_days(self) -> int:
        return 1


class Order:
    """Order with configurable shipping strategy."""
    
    def __init__(self, weight: float, distance: float):
        self.weight = weight
        self.distance = distance
        self._shipping: ShippingStrategy = StandardShipping()
    
    def set_shipping(self, strategy: ShippingStrategy) -> None:
        self._shipping = strategy
    
    def shipping_cost(self) -> float:
        return self._shipping.calculate(self.weight, self.distance)
    
    def estimated_delivery(self) -> int:
        return self._shipping.delivery_days


# Compare shipping options
order = Order(weight=5.0, distance=500)

strategies = [
    ("Standard", StandardShipping()),
    ("Express", ExpressShipping()),
    ("Overnight", OvernightShipping()),
]

print(f"Order: {order.weight}kg, {order.distance}km\n")
print(f"{'Method':<12} {'Cost':>10} {'Delivery':>12}")
print("-" * 36)

for name, strategy in strategies:
    order.set_shipping(strategy)
    cost = order.shipping_cost()
    days = order.estimated_delivery()
    print(f"{name:<12} ${cost:>9.2f} {days:>8} days")

---

# 3. Factory Pattern

## 3.1 Overview

The **Factory** pattern centralizes object creation logic. Instead of calling constructors directly, you use a factory method or class.

### When to Use Factory

| Scenario | Benefit |
|----------|--------|
| Multiple similar classes (Developer, Manager, Intern) | Single creation point |
| Complex construction logic | Hides complexity |
| Creation depends on configuration or input | Flexible instantiation |
| Need to decouple code from specific classes | Easier maintenance |

## 3.2 Example: Employee Factory

## 3.5 Factory Pattern Variations

### Types of Factory Patterns

| Pattern | Description | Use When |
|---------|-------------|----------|
| **Simple Factory** | Single method creates objects | Few types, simple creation |
| **Factory Method** | Subclasses decide what to create | Framework/library design |
| **Abstract Factory** | Creates families of related objects | Platform-specific objects |
| **Registry Factory** | Map of names to classes | Plugin systems, extensibility |

### Simple Factory vs Factory Method

```python
# Simple Factory: One class, one method
class AnimalFactory:
    @staticmethod
    def create(animal_type: str) -> Animal:
        if animal_type == "dog": return Dog()
        if animal_type == "cat": return Cat()

# Factory Method: Subclasses decide
class AnimalFactory(ABC):
    @abstractmethod
    def create_animal(self) -> Animal:
        pass

class DogFactory(AnimalFactory):
    def create_animal(self) -> Animal:
        return Dog()
```

### When to Use Factory Pattern

| Scenario | Without Factory | With Factory |
|----------|----------------|--------------|
| Adding new type | Modify all `if/else` chains | Add class, register |
| Testing | Hard to mock constructors | Inject mock factory |
| Configuration-based creation | Scattered logic | Centralized |
| Complex construction | Duplicated setup code | Single location |

### Factory Pattern Anti-Patterns

| Anti-Pattern | Problem | Solution |
|--------------|---------|----------|
| **God Factory** | One factory creates everything | Split by domain |
| **Over-engineering** | Factory for 1-2 simple classes | Use constructor directly |
| **Leaky abstraction** | Factory exposes internal details | Return interface, not concrete |

### The Registry Pattern Explained

The registry pattern (shown in examples) is powerful because:

1. **Decentralized registration** — Classes can register themselves
2. **Runtime extensibility** — Add types without recompilation
3. **Plugin architecture** — External code can add types

```python
# Self-registering classes using decorators
def register(name: str):
    def decorator(cls):
        Registry.add(name, cls)
        return cls
    return decorator

@register("premium")
class PremiumUser(User):
    pass

# Now PremiumUser is automatically in the registry!
```

In [ ]:
# Employee class hierarchy

class Employee:
    """Base employee class."""
    
    def __init__(self, name: str, employee_id: int):
        self.name = name
        self.employee_id = employee_id

    def role(self) -> str:
        return "Employee"
    
    def annual_bonus(self) -> float:
        return 1000.0
    
    def __repr__(self) -> str:
        return f"{self.role()}(name={self.name}, id={self.employee_id})"


class Developer(Employee):
    def role(self) -> str:
        return "Developer"
    
    def annual_bonus(self) -> float:
        return 3000.0


class Manager(Employee):
    def __init__(self, name: str, employee_id: int, team_size: int = 0):
        super().__init__(name, employee_id)
        self.team_size = team_size
    
    def role(self) -> str:
        return "Manager"
    
    def annual_bonus(self) -> float:
        return 5000.0 + (self.team_size * 500)


class Intern(Employee):
    def role(self) -> str:
        return "Intern"
    
    def annual_bonus(self) -> float:
        return 500.0

## 3.3 Factory with Registry Pattern

The registry pattern maps type names to classes, making the factory extensible:

In [ ]:
class EmployeeFactory:
    """Factory for creating employees with registry pattern."""
    
    # Registry maps type names to classes
    _registry: dict[str, type[Employee]] = {
        "employee": Employee,
        "developer": Developer,
        "manager": Manager,
        "intern": Intern,
    }
    
    # Counter for auto-generating IDs
    _next_id: int = 1000

    @classmethod
    def create(cls, employee_type: str, name: str, **kwargs) -> Employee:
        """Create an employee by type name.
        
        Args:
            employee_type: String like "developer", "manager"
            name: Employee name
            **kwargs: Additional arguments for specific types
        """
        key = employee_type.lower().strip()
        
        if key not in cls._registry:
            available = ", ".join(cls._registry.keys())
            raise ValueError(f"Unknown type '{employee_type}'. Available: {available}")
        
        # Auto-generate ID
        employee_id = cls._next_id
        cls._next_id += 1
        
        # Create instance
        employee_class = cls._registry[key]
        return employee_class(name, employee_id, **kwargs)
    
    @classmethod
    def register(cls, type_name: str, employee_class: type[Employee]) -> None:
        """Register a new employee type (extensibility!)."""
        cls._registry[type_name.lower()] = employee_class
    
    @classmethod
    def available_types(cls) -> list[str]:
        """List available employee types."""
        return list(cls._registry.keys())


# Create employees using the factory
team = [
    EmployeeFactory.create("developer", "Alice Chen"),
    EmployeeFactory.create("developer", "Bob Smith"),
    EmployeeFactory.create("manager", "Carol White", team_size=5),
    EmployeeFactory.create("intern", "Dave Brown"),
]

print("Team members:")
for emp in team:
    print(f"  {emp} - Bonus: ${emp.annual_bonus():.0f}")

print(f"\nAvailable types: {EmployeeFactory.available_types()}")

# Try invalid type
try:
    EmployeeFactory.create("ceo", "Big Boss")
except ValueError as e:
    print(f"\nError: {e}")

## 3.4 Extending the Factory

The registry pattern allows adding new types without modifying factory code:

In [ ]:
# Add new employee type without touching EmployeeFactory code!

class Contractor(Employee):
    """Contractor — temporary employee."""
    
    def __init__(self, name: str, employee_id: int, hourly_rate: float = 50):
        super().__init__(name, employee_id)
        self.hourly_rate = hourly_rate
    
    def role(self) -> str:
        return "Contractor"
    
    def annual_bonus(self) -> float:
        return 0.0  # Contractors don't get bonuses


# Register the new type
EmployeeFactory.register("contractor", Contractor)

# Now we can create contractors!
contractor = EmployeeFactory.create("contractor", "Eve Wilson", hourly_rate=75)
print(f"New hire: {contractor}")
print(f"Hourly rate: ${contractor.hourly_rate}")
print(f"Available types: {EmployeeFactory.available_types()}")

---

# 4. Singleton Pattern

## 4.1 Overview

The **Singleton** pattern ensures a class has only **one instance** and provides global access to it.

### Common Use Cases

| Use Case | Rationale |
|----------|----------|
| Configuration/Settings | Single source of truth |
| Database connection pool | Shared resource |
| Logger | Consistent logging across application |
| Cache | Shared cache instance |

### Trade-offs

| Pros | Cons |
|------|------|
| Single instance guaranteed | Global state (harder to test) |
| Lazy initialization | Hidden dependencies |
| Global access point | Tight coupling |

## 4.2 Implementation Using `__new__`

In [ ]:
# Method 1: Classic Singleton using __new__

class ConfigSingleton:
    """Application configuration — only one instance exists."""
    
    _instance: "ConfigSingleton | None" = None
    _initialized: bool = False

    def __new__(cls, *args, **kwargs) -> "ConfigSingleton":
        if cls._instance is None:
            cls._instance = super().__new__(cls)
        return cls._instance

    def __init__(self, settings: dict | None = None):
        # Only initialize once!
        if ConfigSingleton._initialized:
            return
        
        self.settings: dict = dict(settings or {})
        self._defaults: dict = {
            "debug": False,
            "log_level": "INFO",
            "max_connections": 10,
        }
        ConfigSingleton._initialized = True
    
    def get(self, key: str, default=None):
        return self.settings.get(key, self._defaults.get(key, default))
    
    def set(self, key: str, value) -> None:
        self.settings[key] = value


# Test singleton behavior
config1 = ConfigSingleton({"env": "development", "debug": True})
config2 = ConfigSingleton({"env": "production"})  # Ignored! Already initialized

print(f"Same instance? {config1 is config2}")
print(f"Config env: {config1.get('env')}")  # First initialization wins
print(f"Config debug: {config2.get('debug')}")

# Changes are visible to all "instances"
config1.set("api_key", "secret123")
print(f"API key from config2: {config2.get('api_key')}")

## 4.3 Pythonic Alternative: Module-Level Singleton

In Python, **modules are singletons by nature**. A simpler approach is to use module-level instances:

In [ ]:
# config.py (imagine this is a separate file)
# ------------------------------------------
# SETTINGS = {
#     "env": "development",
#     "debug": True,
# }
#
# def get_setting(key, default=None):
#     return SETTINGS.get(key, default)
#
# def set_setting(key, value):
#     SETTINGS[key] = value
# ------------------------------------------
#
# Usage in other files:
# from config import SETTINGS, get_setting
#
# This is simpler and more Pythonic than a Singleton class!

# Simulating module-level singleton
class _Config:
    """Internal config class."""
    
    def __init__(self):
        self.data: dict = {"env": "development", "debug": True}
    
    def get(self, key: str, default=None):
        return self.data.get(key, default)
    
    def set(self, key: str, value) -> None:
        self.data[key] = value


# Module-level instance (created once when module loads)
config = _Config()

# Usage
print(f"Module singleton — env: {config.get('env')}")
config.set("new_setting", "value")
print(f"Module singleton — new_setting: {config.get('new_setting')}")

## 4.4 When to Avoid Singleton

Consider alternatives when:
- You need testability (singletons are hard to mock)
- The "single instance" requirement may change
- You're hiding dependencies (use dependency injection instead)

**Better alternatives:**
- Dependency injection
- Module-level instances
- Class methods with class-level state

## 4.5 Singleton: The Controversial Pattern

### Why Singleton Is Controversial

Singleton is often called an "anti-pattern" because it can lead to:

| Problem | Description |
|---------|-------------|
| **Global state** | Any code can modify the singleton, causing hidden dependencies |
| **Testing difficulty** | Hard to reset state between tests |
| **Hidden dependencies** | Classes use singleton without declaring the dependency |
| **Concurrency issues** | Multiple threads accessing shared state |
| **Tight coupling** | Code depends on specific singleton class |

### The "Singleton Addiction"

```python
# ❌ Singleton abuse — hidden dependencies everywhere
class UserService:
    def get_user(self, id):
        db = Database.get_instance()      # Hidden dependency
        cache = Cache.get_instance()      # Hidden dependency
        logger = Logger.get_instance()    # Hidden dependency
        # How do you test this? How do you know what it depends on?

# ✅ Explicit dependencies — clear and testable
class UserService:
    def __init__(self, db: Database, cache: Cache, logger: Logger):
        self.db = db
        self.cache = cache
        self.logger = logger
```

### When Singleton IS Appropriate

| Use Case | Rationale |
|----------|-----------|
| **Hardware access** | Only one printer spooler, one GPU manager |
| **Configuration** | App-wide settings loaded once |
| **Connection pools** | Shared, managed resource |
| **Logging** | Consistent across application |

### Singleton Alternatives

| Alternative | How It Works | Benefits |
|-------------|--------------|----------|
| **Dependency Injection** | Pass instance to constructors | Testable, explicit |
| **Module-level instance** | `config = Config()` in module | Pythonic, simple |
| **Borg pattern** | Shared state, multiple instances | More flexible |
| **Service Locator** | Central registry of services | Discoverable (but still global) |

### The Borg Pattern (Python Alternative)

Instead of ensuring one instance, ensure all instances share state:

```python
class Borg:
    _shared_state = {}
    
    def __init__(self):
        self.__dict__ = self._shared_state

# Multiple instances, but they all share the same data
b1 = Borg()
b1.value = 42

b2 = Borg()
print(b2.value)  # 42 — shared!
print(b1 is b2)  # False — different objects, same state
```

### Thread-Safe Singleton

In multi-threaded applications, naive singletons can create multiple instances:

```python
import threading

class ThreadSafeSingleton:
    _instance = None
    _lock = threading.Lock()
    
    def __new__(cls):
        if cls._instance is None:
            with cls._lock:  # Only one thread can enter
                # Double-check after acquiring lock
                if cls._instance is None:
                    cls._instance = super().__new__(cls)
        return cls._instance
```

---

# 5. Exercises

| Exercise | Topics | Difficulty |
|----------|--------|-----------|
| 1 | Strategy Pattern | Basic |
| 2 | Factory Pattern | Intermediate |
| 3 | Composition | Intermediate |
| 4 | Complete System | Advanced |

## Exercise 1: Text Formatter Strategy

Create a text formatting system using the Strategy pattern:

1. `TextFormatter` ABC with `format(text: str) -> str` method
2. Concrete formatters: `UppercaseFormatter`, `LowercaseFormatter`, `TitleCaseFormatter`
3. `TextEditor` class that uses a formatter strategy
4. Ability to change formatter at runtime

In [ ]:
# Exercise 1: Text Formatter Strategy - Starter Code

class TextFormatter(ABC):
    """Strategy interface for text formatting."""
    
    @abstractmethod
    def format(self, text: str) -> str:
        pass


class UppercaseFormatter(TextFormatter):
    # TODO: Implement uppercase formatting
    pass


class LowercaseFormatter(TextFormatter):
    # TODO: Implement lowercase formatting
    pass


class TitleCaseFormatter(TextFormatter):
    # TODO: Implement title case formatting
    pass


class TextEditor:
    """Context that uses a text formatter strategy."""
    
    def __init__(self, formatter: TextFormatter | None = None):
        # TODO: Initialize with formatter (default to no formatting)
        pass
    
    def set_formatter(self, formatter: TextFormatter) -> None:
        # TODO: Change formatter
        pass
    
    def edit(self, text: str) -> str:
        # TODO: Apply formatter to text
        pass


# === Demo (uncomment after implementation) ===
# editor = TextEditor(UppercaseFormatter())
# print(editor.edit("hello world"))  # HELLO WORLD
#
# editor.set_formatter(TitleCaseFormatter())
# print(editor.edit("hello world"))  # Hello World

## Exercise 2: Vehicle Factory

Create a factory for different vehicle types:

- `Vehicle` base class with `brand`, `model`, and `wheels` attributes
- Subclasses: `Car` (4 wheels), `Motorcycle` (2 wheels), `Truck` (6 wheels)
- `VehicleFactory` with registry pattern
- Factory should auto-generate IDs

**Bonus:** Add a `register()` method to allow adding new vehicle types.

In [ ]:
# Exercise 2: Vehicle Factory - Starter Code

class Vehicle:
    """Base vehicle class."""
    wheels: int = 0
    
    def __init__(self, brand: str, model: str, vehicle_id: int):
        self.brand = brand
        self.model = model
        self.vehicle_id = vehicle_id
    
    def describe(self) -> str:
        return f"{self.brand} {self.model} ({self.wheels} wheels)"
    
    def __repr__(self) -> str:
        return f"{self.__class__.__name__}(id={self.vehicle_id}, {self.describe()})"


class Car(Vehicle):
    # TODO: Set wheels = 4
    pass


class Motorcycle(Vehicle):
    # TODO: Set wheels = 2
    pass


class Truck(Vehicle):
    # TODO: Set wheels = 6
    pass


class VehicleFactory:
    """Factory for creating vehicles."""
    
    _registry: dict[str, type[Vehicle]] = {
        # TODO: Map "car" -> Car, "motorcycle" -> Motorcycle, "truck" -> Truck
    }
    _next_id: int = 1
    
    @classmethod
    def create(cls, vehicle_type: str, brand: str, model: str) -> Vehicle:
        # TODO: 
        # 1. Normalize vehicle_type (lowercase, strip)
        # 2. Check if type exists in registry
        # 3. Get next ID and increment counter
        # 4. Create and return vehicle instance
        pass
    
    @classmethod
    def register(cls, type_name: str, vehicle_class: type[Vehicle]) -> None:
        # TODO: Add new type to registry
        pass


# === Demo (uncomment after implementation) ===
# fleet = [
#     VehicleFactory.create("car", "Toyota", "Camry"),
#     VehicleFactory.create("car", "Honda", "Civic"),
#     VehicleFactory.create("motorcycle", "Harley", "Sportster"),
#     VehicleFactory.create("truck", "Ford", "F-150"),
# ]
#
# print("Fleet:")
# for vehicle in fleet:
#     print(f"  {vehicle}")

## Exercise 3: Document with Exporters (Composition)

Build a document system using composition:

- `Document` class with `title` and `content`
- `DocumentExporter` ABC with `export(doc)` method
- Concrete exporters: `MarkdownExporter`, `HTMLExporter`, `PlainTextExporter`
- Document can change its exporter at runtime

This exercise demonstrates the principle "favor composition over inheritance."

In [ ]:
# Exercise 3: Document with Exporters - Starter Code

class DocumentExporter(ABC):
    """Abstract base for document exporters."""
    
    @abstractmethod
    def export(self, title: str, content: str) -> str:
        """Export document to string format."""
        pass


class PlainTextExporter(DocumentExporter):
    def export(self, title: str, content: str) -> str:
        # TODO: Return plain text format
        # Title
        # -----
        # Content
        pass


class MarkdownExporter(DocumentExporter):
    def export(self, title: str, content: str) -> str:
        # TODO: Return Markdown format
        # # Title
        # 
        # Content
        pass


class HTMLExporter(DocumentExporter):
    def export(self, title: str, content: str) -> str:
        # TODO: Return HTML format
        # <h1>Title</h1><p>Content</p>
        pass


class Document:
    """Document with swappable export format."""
    
    def __init__(self, title: str, content: str, exporter: DocumentExporter | None = None):
        self.title = title
        self.content = content
        self.exporter = exporter or PlainTextExporter()
    
    def export(self) -> str:
        # TODO: Use self.exporter to export
        pass
    
    def set_exporter(self, exporter: DocumentExporter) -> None:
        # TODO: Change the exporter
        pass


# === Demo (uncomment after implementation) ===
# doc = Document("My Report", "This is the report content.")
#
# print("=== Plain Text ===")
# print(doc.export())
#
# doc.set_exporter(MarkdownExporter())
# print("\n=== Markdown ===")
# print(doc.export())
#
# doc.set_exporter(HTMLExporter())
# print("\n=== HTML ===")
# print(doc.export())

## Exercise 4: Library Management System

Build a comprehensive library system combining multiple design patterns:

**1. Book class** with properties:
- `isbn` (read-only)
- `title` (read-only)
- `available` (can be changed)

**2. Member class** with:
- `member_id`, `name`
- `borrowed_books` list
- `borrow(book)` and `return_book(book)` methods

**3. Library class** (composition):
- Uses a `Storage` implementation for persistence
- `add_book(book)`, `register_member(member)`
- `checkout(member_id, isbn)`, `return_book(member_id, isbn)`
- `get_available_books()`

**4. Optional:** Add a `BookFactory` for creating books from different sources

This exercise integrates composition, strategy, and factory patterns.

In [ ]:
# Exercise 4: Library Management System - Starter Code

# Storage interface (from Unit 8a)
class Storage(ABC):
    @abstractmethod
    def save(self, data) -> None:
        pass
    
    @abstractmethod
    def load(self):
        pass


class MemoryStorage(Storage):
    def __init__(self):
        self._data = None
    
    def save(self, data) -> None:
        self._data = data
    
    def load(self):
        return self._data


class Book:
    """Book with encapsulated attributes."""
    
    def __init__(self, isbn: str, title: str, author: str):
        self._isbn = isbn
        self._title = title
        self._author = author
        self._available = True
    
    @property
    def isbn(self) -> str:
        # TODO: Return ISBN (read-only)
        pass
    
    @property
    def title(self) -> str:
        # TODO: Return title (read-only)
        pass
    
    @property
    def author(self) -> str:
        # TODO: Return author (read-only)
        pass
    
    @property
    def available(self) -> bool:
        # TODO: Return availability
        pass
    
    @available.setter
    def available(self, value: bool) -> None:
        # TODO: Set availability (boolean)
        pass
    
    def __repr__(self) -> str:
        status = "Available" if self._available else "Checked Out"
        return f"Book('{self._title}' by {self._author}, {status})"


class Member:
    """Library member who can borrow books."""
    
    def __init__(self, member_id: str, name: str):
        self.member_id = member_id
        self.name = name
        self._borrowed_books: list[Book] = []
    
    @property
    def borrowed_books(self) -> list[Book]:
        """Return copy of borrowed books list."""
        return list(self._borrowed_books)
    
    def borrow(self, book: Book) -> None:
        # TODO: Add book to borrowed list if available
        # Mark book as unavailable
        # Raise ValueError if book not available
        pass
    
    def return_book(self, book: Book) -> None:
        # TODO: Remove book from borrowed list
        # Mark book as available
        # Raise ValueError if book not in borrowed list
        pass
    
    def __repr__(self) -> str:
        return f"Member({self.member_id}, {self.name}, {len(self._borrowed_books)} books)"


class Library:
    """Library using composition for storage."""
    
    def __init__(self, name: str, storage: Storage | None = None):
        self.name = name
        self._books: dict[str, Book] = {}      # isbn -> Book
        self._members: dict[str, Member] = {}  # member_id -> Member
        self._storage = storage or MemoryStorage()
    
    def add_book(self, book: Book) -> None:
        # TODO: Add book to library
        pass
    
    def register_member(self, member: Member) -> None:
        # TODO: Register member
        pass
    
    def checkout(self, member_id: str, isbn: str) -> str:
        # TODO: 
        # 1. Find member and book
        # 2. Call member.borrow(book)
        # 3. Return success message
        pass
    
    def return_book(self, member_id: str, isbn: str) -> str:
        # TODO:
        # 1. Find member and book
        # 2. Call member.return_book(book)
        # 3. Return success message
        pass
    
    def get_available_books(self) -> list[Book]:
        # TODO: Return list of available books
        pass


# === Demo (uncomment after implementation) ===
# # Create library
# library = Library("City Library")
#
# # Add books
# library.add_book(Book("978-0-13-468599-1", "The Pragmatic Programmer", "Hunt & Thomas"))
# library.add_book(Book("978-0-596-51774-8", "JavaScript: The Good Parts", "Douglas Crockford"))
# library.add_book(Book("978-1-49-195017-1", "Fluent Python", "Luciano Ramalho"))
#
# # Register members
# library.register_member(Member("M001", "Alice"))
# library.register_member(Member("M002", "Bob"))
#
# # Checkout and return
# print(library.checkout("M001", "978-0-13-468599-1"))
# print(f"Available books: {library.get_available_books()}")
#
# print(library.return_book("M001", "978-0-13-468599-1"))
# print(f"Available books: {library.get_available_books()}")

---

## 📝 Summary

This unit covered essential design patterns for object-oriented programming:

**🏗️ Composition vs Inheritance**
- Use inheritance for "is-a" relationships (Dog IS an Animal)
- Use composition for "has-a" relationships (Car HAS an Engine)
- Prefer composition for flexibility and loose coupling
- Composition allows runtime behavior changes

**🎯 Strategy Pattern**
- Encapsulate interchangeable algorithms in separate classes
- Define a common interface for all strategies
- Context class delegates work to strategy object
- Change behavior at runtime by swapping strategies

**🏭 Factory Pattern**
- Centralize object creation logic
- Use registry pattern for extensibility
- Decouple client code from concrete classes
- Easy to add new types without modifying factory

**🔒 Singleton Pattern**
- Ensure only one instance exists
- Useful for configuration, logging, caching
- Consider trade-offs: global state, testing difficulty
- Pythonic alternative: module-level instances

---

**Congratulations!** You have completed Unit 8 — Advanced OOP & Design Principles.

**Next:** Unit 9 — Computational Thinking & Algorithms

---

## 🔗 How These Patterns Work Together

Design patterns are often combined to create robust architectures:

### Common Pattern Combinations

| Combination | How They Work Together |
|-------------|----------------------|
| **Factory + Strategy** | Factory creates strategy objects based on configuration |
| **Singleton + Factory** | Single factory instance manages object creation |
| **Strategy + Composition** | Context composed with strategy objects |
| **Factory + Registry** | Registry enables extensible factories |

### Example: Complete E-Commerce System

```python
# Factory creates payment strategies
payment = PaymentFactory.create("credit_card", card_info)

# Strategy processes the payment
cart.set_payment_strategy(payment)

# Singleton logger records the transaction
Logger.instance().log(f"Payment processed: {cart.total()}")

# Composition: Cart HAS a payment strategy, HAS items
```

### Choosing the Right Pattern

```
┌─────────────────────────────────────────────────────────┐
│              Pattern Selection Guide                     │
├─────────────────────────────────────────────────────────┤
│                                                          │
│  Need to swap algorithms at runtime?                    │
│       └── ✅ Strategy Pattern                           │
│                                                          │
│  Need to create objects without specifying class?       │
│       └── ✅ Factory Pattern                            │
│                                                          │
│  Need exactly one instance of a class?                  │
│       └── ⚠️  Singleton (consider alternatives first)   │
│                                                          │
│  Need flexible object relationships?                    │
│       └── ✅ Composition over Inheritance               │
│                                                          │
│  Need to add behavior without modifying classes?        │
│       └── ✅ Decorator Pattern (see further reading)    │
│                                                          │
│  Need to notify multiple objects of changes?            │
│       └── ✅ Observer Pattern (see further reading)     │
│                                                          │
└─────────────────────────────────────────────────────────┘
```

### Further Reading: Other Important Patterns

These patterns are also worth studying:

| Pattern | Purpose | Example |
|---------|---------|---------|
| **Observer** | Notify dependents of state changes | Event systems, UI updates |
| **Decorator** | Add behavior dynamically | Logging, caching wrappers |
| **Adapter** | Make incompatible interfaces work together | Legacy system integration |
| **Facade** | Simplify complex subsystems | API wrapper classes |
| **Command** | Encapsulate requests as objects | Undo/redo, task queues |
| **Template Method** | Define algorithm skeleton, let subclasses fill in | Frameworks, hooks |